# Regional Error Detection at the Californian Coast

This notebook will plot, analyze, and clarify errors between satellite and saildrone data for sea surface salinity (SSS) and sea surface temperature (SST).

**When, where, why are differences largest?**

### Part 1. Select a region and aggregate data

We will chart data on the california coast.

In [14]:
import sys
from colorama import Fore
from colorama import Back

import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates

#load map packages
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from calendar import month_abbr

In [15]:
%run ../"Project 3 - Clean Anomalies"/"CleanAnomalies-SingleFunction.ipynb"
sail = cleanAnomalies("../../saildrone_data/saildrone_west_coast_survey_2019_sd1044_JPLv5.0_8dy_20210613norep_20210613_20211116.nc")
sail

FileNotFoundError: [Errno 2] No such file or directory: b'/home/brazil-0034/2022_SummerInternship_FarallonInstitute/saildrone_west_coast_survey_2019_sd1044_JPLv5.0_8dy_20210613norep_20210613_20211116.nc'

sat_smap_sss - satellite

SAL_CTD_MEAN - saildrone

#### Detect outliers and set constraints

**For Standard Deviation Method (Uncomment Bottom Box):**
- Range (stDevRange) is the number of standard deviations from the mean to consider an outlier.
- The default is 1 and presents a good handful of clear outliers in the data.
- However, many statistical samples generally consider the range for a valid outlier to be 2.5 to 3 standard deviations from the mean.
- Here, one standard deviation is plenty to represent the differences in the data - because the innacuracies are quite visible already.

In [ ]:
satData = sail['sat_smap_sss'] # change to sat_anc_sst for temperature
seaData = sail['SAL_CTD_MEAN'] # change to analyzed_sst for temperature

quantileRange = 0.1

qt = len(seaData)

# we store the data in a 2d array to later rebuild into a dataframe
data = [[], [], [], []]

for x in range(qt):
    diff = abs(satData[x] - seaData[x])
    data[0].append(float(sail.lon[x]))
    data[1].append(float(sail.lat[x]))
    data[2].append(float(diff))
    data[3].append(x) # index in sail[]

differences = pd.DataFrame({'lon': data[0], 'lat': data[1], 'diff': data[2]})
    
# filter out based on quartiles
less = differences.quantile(0 + quantileRange)[2]
more = differences.quantile(1 - quantileRange)[2]
print(str(less) + ", " + str(more))

safeValues = 0
for x in range(qt):
    diff = differences['diff'][x]
    if (diff < more and diff > less): # if our number is within the safe range
        # do nun, it's safe!
        differences = differences.drop(x) # we DROP IT!
        print("Dropped value " + str(diff) + " out of DataFrame")
    else: # if it isn't
        safeValues += 1
        
# sort the index so they go from 0-xxx without any holes
differences = differences.reset_index(drop = True)

In [ ]:
# stDevRange = 1

# stDev = float(sail['sat_smap_sss'].std()) * stDevRange

# satData = sail['sat_smap_sss'] # change to sat_anc_sst for temperature
# seaData = sail['SAL_CTD_MEAN'] # change to analyzed_sst for temperature

# qt = len(seaData)

# # we store the data in a 2d array to later rebuild into a dataframe
# data = [[], [], [], []]

# for x in range(qt):
#     diff = abs(satData[x] - seaData[x])
#     if (diff > stDev):
#         print("Outlier with Diff " + str(float(diff)))
#         data[0].append(float(sail.lon[x]))
#         data[1].append(float(sail.lat[x]))
#         data[2].append(float(diff))
#         data[3].append(x) # index in sail[]

# differences = pd.DataFrame({'lon': data[0], 'lat': data[1], 'diff': data[2]})
# differences

### Part 2. Plotting the Differences Data

In [ ]:
fig = plt.figure(figsize=(16,10), dpi = 72)
ax = plt.axes(projection=ccrs.PlateCarree())

#define latitude and longitude boundaries
latr = [np.min(differences['lat']), np.max(differences['lat'])] 
lonr = [np.max(differences['lon']), np.min(differences['lon'])] 

# Select a region of our data, giving it a margin
margin = 0.5 
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]+margin,lonr[1]-margin]]) 

states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

ax.coastlines(resolution='10m',linewidth=1,color='black')
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.add_feature(cfeature.BORDERS, color = 'black')
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.gridlines(linestyle = '--', linewidth = 0.5)

#plot diff data
plt.title("Greatest Outliers Plot")
plt.scatter(differences.lon, differences.lat, c = differences['diff'])

plt.show()

### Quickly summarize some of our new data

In [ ]:
diffs = differences['diff']

greatest = sys.float_info.min;
least = sys.float_info.max;
total = 0
for x in range(len(diffs)):
    if (diffs[x] > greatest):
        greatest = diffs[x]
    if (diffs[x] < least):
        least = diffs[x]
    total += diffs[x]
    
avg = total / len(diffs)
print(Back.RESET + "Least significant outlier differed by " + Back.RED + str(least))
print(Back.RESET + "Most significant outlier differed by " + Back.GREEN + str(greatest))
print(Back.RESET + "Range of differences was " + Back.YELLOW + str(greatest - least))
print(Back.RESET + "Average difference " + Back.BLUE + str(avg))

### Plot and compare against similar data in the same regions.

#### Leading Questions:
- Do they face the same innacuracies?
- What causes these differences?
- What relationships can we build between SSS/SST and other variables?

#### **In this simple example:**
##### We will plot the significance of a data innacuracy with the distance to land, to see if there is a correlation.

In [ ]:
# xAxis will be distance to land
# yAxis will be difference level
xSample = []
xSampleName = "Distance To Land (km)"

for x in range(len(diffs)):
    indexInSail = data[3][x]
    xSample.append(sail['dist_land'][indexInSail])

#create plot 
plt.figure()
plt.scatter(x = xSample, y = diffs) #seaborn has built in confidence intervals
# make temp/sal plot
#add labels

plt.grid(zorder=0, alpha=0.5)
plt.xlabel(xSampleName)
plt.ylabel('SAT/DRONE DIFFERENCE (SSS in PPU)')
plt.show()

### Summary (none of this is true using the quantiles method and idk why)
#### There is a clear moderately strong, negative relationship between the sat/drone data difference and the distance to land.
##### This demonstrates that there are far more innacuracies when data is measured close to land than when data is measured far from land.
##### This conclusion is supported by the generalization that satellite salinity measurements are generally innacurate when taken close to land.